In [1]:
import numpy as np
from scipy.stats import norm

# needed for compact printing of numpy arrays
np.set_printoptions(suppress=True,precision=4,linewidth=np.inf) # use precision to set the number of decimal digits to display

In [2]:
"""
Authors: Thomas Sargent, John Stachurski

Discretizes Gaussian linear AR(1) processes via Tauchen's method

"""
def approx_markov(rho, sigma_u, m=3, n=7):
    """
    Computes the Markov matrix associated with a discretized version of
    the linear Gaussian AR(1) process

        y_{t+1} = rho * y_t + u_{t+1}

    according to Tauchen's method.  Here {u_t} is an iid Gaussian
    process with zero mean.

    Parameters
    ----------
    rho : scalar(float)
        The autocorrelation coefficient
    sigma_u : scalar(float)
        The standard deviation of the random process
    m : scalar(int), optional(default=3)
        The number of standard deviations to approximate out to
    n : scalar(int), optional(default=7)
        The number of states to use in the approximation

    Returns
    -------

    x : array_like(float, ndim=1)
        The state space of the discretized process
    P : array_like(float, ndim=2)
        The Markov transition matrix where P[i, j] is the probability
        of transitioning from x[i] to x[j]

    """
    F = norm(loc=0, scale=sigma_u).cdf

    # standard deviation of y_t
    std_y = np.sqrt(sigma_u**2 / (1-rho**2))

    # top of discrete state space
    x_max = m * std_y

    # bottom of discrete state space
    x_min = - x_max

    # discretized state space
    x = np.linspace(x_min, x_max, n)

    step = (x_max - x_min) / (n - 1)
    half_step = 0.5 * step
    P = np.empty((n, n))

    for i in range(n):
        P[i, 0] = F(x[0]-rho * x[i] + half_step)
        P[i, n-1] = 1 - F(x[n-1] - rho * x[i] - half_step)
        for j in range(1, n-1):
            z = x[j] - rho * x[i]
            P[i, j] = F(z + half_step) - F(z - half_step)

    return x, P


In [3]:
rho=0.95
sigmae=0.01
N=7
m=3
x, P = approx_markov(rho,sigmae,m,N)
print("Transition Matrix:\n", P)
print("Discretized State Space:\n", x)

Transition Matrix:
 [[0.8688 0.1312 0.     0.     0.     0.     0.    ]
 [0.0273 0.8726 0.1001 0.     0.     0.     0.    ]
 [0.     0.0391 0.8861 0.0748 0.     0.     0.    ]
 [0.     0.     0.0547 0.8907 0.0547 0.     0.    ]
 [0.     0.     0.     0.0748 0.8861 0.0391 0.    ]
 [0.     0.     0.     0.     0.1001 0.8726 0.0273]
 [0.     0.     0.     0.     0.     0.1312 0.8688]]
Discretized State Space:
 [-0.0961 -0.0641 -0.032   0.      0.032   0.0641  0.0961]
